In [1]:
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'



query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
query = query.replace(' ', '+')


news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')


news_dict = {}
idx = 0
cur_page = 1

print()
print('크롤링 중...')

while idx < news_num:
### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###
    
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        idx += 1

    cur_page += 1

    pages = soup.find('div', {'class' : 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')
    
    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')

print('크롤링 완료')

print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
os.startfile(folder_path)

검색 키워드를 입력하세요 : 자동차
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 5

크롤링 중...
크롤링 완료
데이터프레임 변환
엑셀 저장 완료 | 경로 : C:\Users\B-07\네이버뉴스_자동차_2022-03-07_15시55분.xlsx


In [2]:
import requests
from bs4 import BeautifulSoup
import time

In [5]:
url = 'https://search.naver.com/search.naver?where=news&query=%EC%9E%90%EB%8F%99%EC%B0%A8%20%EC%82%AC%EA%B3%A0&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.03.01&de=2022.03.07&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210301to20220307&is_sug_officeid=0'
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')

In [6]:
soup.find_all('a', attrs={'class':'news_tit'})

[<a class="news_tit" href="http://www.newsis.com/view/?id=NISX20220306_0001783567&amp;cID=13001&amp;pID=13000" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=1&amp;i=88000127_000000000000000011046240&amp;g=003.0011046240&amp;u='+urlencode(this.href));" target="_blank" title="사흘만에 또…현대제철, 20대 근로자 사망 사고">사흘만에 또…현대제철, 20대 근로자 사망 <mark>사고</mark></a>,
 <a class="news_tit" href="http://www.segye.com/content/html/2022/03/07/20220307507304.html?OutUrl=naver" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=5&amp;i=880000D2_000000000000000003673665&amp;g=022.0003673665&amp;u='+urlencode(this.href));" target="_blank" title="잇단 근로자 사망사고… 고용부·경찰, 현대제철 등 4곳 압수수색">잇단 근로자 사망<mark>사고</mark>… 고용부·경찰, 현대제철 등 4곳 압수수색</a>,
 <a class="news_tit" href="http://www.enewstoday.co.kr/news/articleView.html?idxno=1553171" onclick="return goOtherCR(this, 'a=nws*b.tit&amp;r=9&amp;i=880003FD_000000000000000000709410&amp;g=5038.0000709410&amp;u='+urlencode(this.href));" target="_blank" title="현대자동차 중고차사업 방향 최초 공

In [7]:
[title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'}) ]

['사흘만에 또…현대제철, 20대 근로자 사망 사고',
 '잇단 근로자 사망사고… 고용부·경찰, 현대제철 등 4곳 압수수색',
 '현대자동차 중고차사업 방향 최초 공개',
 '급발진 사고 차량 결함 증명 하려면?',
 "현대차, '자동차용품+생활용품' 통합 쇼핑몰 '현대Shop' 오픈",
 '현대제철, 예산공장서 근로자 사망 사고 발생…나흘만에 또',
 '캐롯손보 퍼마일자동차보험 출시 2년 만 50만건 돌파',
 '현대제철 예산공장 근로자 사망사고…책임은 누가?',
 '교통사고 후유증, 입원치료로 회복 앞당겨야',
 '서귀포시, 위미항구로 회전교차로 설치…교통사고 예방 효과 탁월']

In [8]:
soup.find_all('span', attrs={'class':'info'})

[<span class="info">11시간 전</span>,
 <span class="info">4시간 전</span>,
 <span class="info">6시간 전</span>,
 <span class="info">2시간 전</span>,
 <span class="info">6시간 전</span>,
 <span class="info">18시간 전</span>,
 <span class="info">7시간 전</span>,
 <span class="info">7시간 전</span>,
 <span class="info">43분 전</span>,
 <span class="info">4시간 전</span>]

In [14]:
dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})]

import re

date_list = []
for date in dates:
    if re.search(r'\d+.\d+.\d+.', date) != None:
        date_list.append(date)

date_list

[]

In [15]:
start = 1
result_df = pd.DataFrame()

while True:
    try:
        url = 'https://search.naver.com/search.naver?where=news&query=%EC%9E%90%EB%8F%99%EC%B0%A8%20%EC%82%AC%EA%B3%A0&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.03.01&de=2022.03.07&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210301to20220307&is_sug_officeid=0'.format(start)
        headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36',
          'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        news_title = [title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'})] # 기사 제목
        news_url = [ url['href'] for url in soup.find_all('a', attrs={'class':'news_tit'}) ] # 기사 url
        
        dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
        news_date = []
        for date in dates:
            if re.search(r'\d+.\d+.\d+.', date) != None: # 기사 작성일 정제
                news_date.append(date)
        
        df = pd.DataFrame({'기사작성일':news_date,'기사제목':news_title,'기사주소':news_url})
        result_df = pd.concat([result_df, df], ignore_index=True)
        start += 10
    
    except: # 오류발생시 몇 페이지까지 크롤링했는지 page를 확인하기 
        print(start)
        break

In [35]:
import requests
from bs4 import BeautifulSoup
firms = ["서울교통"]
for firm in firms:
    url = f"https://search.naver.com/search.naver?query={firm}&where=news&ie=utf8&sm=nws_hty"
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    soup_tit = soup.find_all("a", attrs={"class": "news_tit"})
    soup_info = soup.find_all("span", attrs={"class": "info"})
#     dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
    print(f"{firm} 기사")
    for tit, info in zip(soup_tit, soup_info):
        if firm in tit.get_text():
#             print(f"{tit.get_text()} {info.get_text()} {tit['href']}")
            print(tit.get_text())
            print(info.get_text())
            print(tit['href'])



서울교통 기사
서울교통공사, 추억의 지하철 사진·창립 5주년 축하 그림 공모
7시간 전
http://www.newspim.com/news/view/20220307000240
[B tv 서울뉴스] 서울교통공사, 지하철 승객 부주의 사고는 '치료비 보상 불가...
10시간 전
http://ch1.skbroadband.com/content/view?parent_no=24&content_no=57&p_no=138105
서울교통공사 노·사 공동으로 지하철 현장 안전점검
3시간 전
https://www.nocutnews.co.kr/news/5719307


In [51]:
import requests
from bs4 import BeautifulSoup

firms = ["교통"]
for firm in firms:
    url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={firms}"
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    soup_tit = soup.find_all("a", attrs={"class": "news_tit"})
    soup_info = soup.find_all("span", attrs={"class": "info"})
    soup_img =  soup.find_all("img",attrs = {"class":"thumb api_get"})
#     dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
    indx = 1
    for tit, time,img in zip(soup_tit, soup_info,soup_img):
        if firm in tit.get_text():
#             print(f"{tit.get_text()} {info.get_text()} {tit['href']}")
            print(tit.get_text())
            print(time.get_text())
            print(tit['href'])
            print(img['src'])
            indx +=1
            urllib.request.urlretrieve(img['src'], "이미지{}.jpg".format(indx))

경찰 범죄분석 시스템에 티맵 교통정보 탑재
4시간 전
http://yna.kr/AKR20220304140600004?did=1195m
https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F001%2F2022%2F03%2F07%2F13035840.jpg&type=ff264_180&expire=2&refresh=true
익산시, 교통·물류 인프라 구축 스마트도시 도약
9시간 전
http://www.newsis.com/view/?id=NISX20220307_0001784146&cID=10808&pID=10800
https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F003%2F2022%2F03%2F07%2F11046915.jpg&type=ff264_180&expire=2&refresh=true
도로교통공단, 공공데이터 제공 운영 우수기관 선정
3시간 전
http://autotimes.hankyung.com/apps/news.sub_view?popup=0&nid=03&c1=03&c2=03&c3=00&nkey=202203071639381
https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F5029%2F2022%2F03%2F07%2F84775.jpg&type=ff264_180&expire=2&refresh=true
'시내버스비 왕복 2400원' 보은군 농촌 통학생 교통비 지원
9시간 전
https://www.news1.kr/articles/?4607484
https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F421%2F2022%2F03

In [39]:
import requests
from bs4 import BeautifulSoup

firms = ["서울교통"]
for firm in firms:
    url = f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={firms}"
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    soup_tit = soup.find_all("a", attrs={"class": "news_tit"})
    soup_info = soup.find_all("span", attrs={"class": "info"})
#     dates = [ date.get_text() for date in soup.find_all('span', attrs={'class':'info'})] # 기사 작성일
    print(f"{firm} 기사")
    for tit, time in zip(soup_tit, soup_info):
        if firm in tit.get_text():
#             print(f"{tit.get_text()} {info.get_text()} {tit['href']}")
            print(tit.get_text())
            print(time.get_text())
            print(tit['href'])
            urllib.request.urlretrieve(product_img_url, "이미지.jpg")

서울교통 기사
서울교통공사, 추억의 지하철 사진·창립 5주년 축하 그림 공모
8시간 전
http://www.newspim.com/news/view/20220307000240
[B tv 서울뉴스] 서울교통공사, 지하철 승객 부주의 사고는 '치료비 보상 불가...
8시간 전
http://ch1.skbroadband.com/content/view?parent_no=24&content_no=57&p_no=138105
서울교통공사 노·사 공동으로 지하철 현장 안전점검
1일 전
https://www.nocutnews.co.kr/news/5719307


In [45]:
import requests
from bs4 import BeautifulSoup
import urllib.request

URL = 'https://www.coupang.com/vp/products/5647481827?itemId=9231892317&vendorItemId=76517595808&sourceType=CAMPAIGN&campaignId=82&categoryId=0&isAddedCart='

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}
res = requests.get(URL, headers=headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml") 

product_img_url = "http:"+ soup.select_one('#repImageContainer > img')['src']

urllib.request.urlretrieve(product_img_url, "이미지.jpg")

('이미지.jpg', <http.client.HTTPMessage at 0x1d997912e20>)